In [19]:
import pdfplumber
import pymongo
import pandas as pd
from pymongo import MongoClient
import pandas as pd


In [2]:
"""def extract_dimensions(pdf_path):
    dimensions = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            # Look for tables in each page
            tables = page.extract_tables()
            for table in tables:
                # Process each table row into a dictionary
                headers = ["Dim", "Description", "Comment", "Tol (-)", "Tol (+)", "XS", "S", "M", "L", "XL"]
                for row in table:
                    # if len(row) == len(headers):  # Ensure the row matches headers
                    #     print("u")
                    print(row)
                    row_data = dict(zip(headers, row))
                    dimensions.append(row_data)
    return dimensions"""


'def extract_dimensions(pdf_path):\n    dimensions = []\n    with pdfplumber.open(pdf_path) as pdf:\n        for page in pdf.pages:\n            # Look for tables in each page\n            tables = page.extract_tables()\n            for table in tables:\n                # Process each table row into a dictionary\n                headers = ["Dim", "Description", "Comment", "Tol (-)", "Tol (+)", "XS", "S", "M", "L", "XL"]\n                for row in table:\n                    # if len(row) == len(headers):  # Ensure the row matches headers\n                    #     print("u")\n                    print(row)\n                    row_data = dict(zip(headers, row))\n                    dimensions.append(row_data)\n    return dimensions'

In [20]:
def extract_dimensions(pdf_path):
    dimensions = []
    with pdfplumber.open(pdf_path) as pdf:
        for page in pdf.pages:
            tables = page.extract_tables()
            for table in tables:
                if len(table) > 0:
                    # Use the first row of the table as the header
                    headers = table[0]
                    for row in table[1:]:
                        if len(row) == len(headers):  # Only process rows matching header length
                            # Ensure keys are strings and handle None values
                            row_data = dict(zip(headers, row))
                            valid_row_data = {
                                str(k) if k is not None else "Unknown": v
                                for k, v in row_data.items()
                            }
                            dimensions.append(valid_row_data)
    return dimensions


In [21]:
def store_in_mongo(data, db_name, collection_name):
    # Ensure all keys are strings
    valid_data = [
        {str(k) if k is not None else "Unknown": v for k, v in doc.items()}
        for doc in data
    ]

    # Connect to MongoDB
    client = pymongo.MongoClient("mongodb://localhost:27017/")
    db = client[db_name]
    collection = db[collection_name]
    
    # Insert data
    collection.insert_many(valid_data)
    print(f"Inserted {len(valid_data)} records into {collection_name} collection in {db_name} database.")


In [22]:
def main():
    pdf_path = r"E:\git\Automated-Email-Parsing-and-Document-Generation\input_doc.pdf"  # Path to the input PDF document
    dimensions_data = extract_dimensions(pdf_path)
    print(dimensions_data)
    # MongoDB storage
    db_name = "technical_specs"
    collection_name = "dimensions"
    store_in_mongo(dimensions_data, db_name, collection_name)

if __name__ == "__main__":
    main()


[{'Dim': '1B1', 'Description': 'ChestWidth', 'Comment': '', 'Tol\n(-)': '-1.00', 'Tol\n(+)': '1.00', 'XS': 'Increm\nent', 'Unknown': 'Deviation', 'S': 'Incremen\nt'}, {'Dim': '1X1', 'Description': 'BottomHem Width', 'Comment': '', 'Tol\n(-)': '-1.00', 'Tol\n(+)': '1.00', 'XS': '46.00', 'Unknown': '', 'S': '50.00'}, {'Dim': '1F3', 'Description': 'AcrossShoulder', 'Comment': '', 'Tol\n(-)': '-1.00', 'Tol\n(+)': '1.00', 'XS': '36.00', 'Unknown': '', 'S': '38.00'}, {'Dim': '1F6', 'Description': 'ShoulderAngle', 'Comment': '', 'Tol\n(-)': '-0.50', 'Tol\n(+)': '0.50', 'XS': '2.50', 'Unknown': '', 'S': '2.50'}, {'Dim': '1G5', 'Description': 'SleeveLength (Short)', 'Comment': '', 'Tol\n(-)': '-1.00', 'Tol\n(+)': '1.00', 'XS': '37.00', 'Unknown': '', 'S': '39.00'}, {'Dim': '1H2', 'Description': 'ArmholeDepth', 'Comment': '', 'Tol\n(-)': '-0.50', 'Tol\n(+)': '0.50', 'XS': '21.20', 'Unknown': '', 'S': '22.00'}, {'2A1': '2K1', 'Upper ArmWidth': 'SleeveHem Width', '': '', '-0.50': '-0.50', '0.50': 

In [23]:
# Connect to MongoDB
from pymongo import MongoClient
import pandas as pd

client = MongoClient("mongodb://localhost:27017/")
db = client["technical_specs"]
collection = db["dimensions"]

# Fetch all records
records = list(collection.find())

# Extract unique column names dynamically from MongoDB records
columns = set()
for record in records:
    columns.update(record.keys())

columns = list(columns)
columns.remove("_id")  # Remove MongoDB's default '_id' field if present
columns.sort()  # Optional: Sort the columns for better readability

print("Extracted Columns:", columns)


Extracted Columns: ['', '-0.50', '0.50', '17.00', '18.00', '2A1', 'Comm\nent', 'Comment', 'Composition', 'Description', 'Dim', 'L', 'M', 'Placement', 'Qty', 'S', 'Tol\n(+)', 'Tol\n(-)', 'Tol (-)', 'Tol(+)', 'Unknown', 'Upper ArmWidth', 'XL', 'XS']


In [24]:
# Prepare a DataFrame using the extracted columns
df = pd.DataFrame(records)

# Remove the MongoDB '_id' column if it exists
if "_id" in df.columns:
    df = df.drop(columns=["_id"])

# Display the first few rows to verify the structure
df.head()


,Dim,Description,Comment,Tol\n(-),Tol\n(+),XS,Unknown,S,2A1,Upper ArmWidth,...,18.00,M,L,XL,Comm\nent,Tol (-),Tol(+),Placement,Qty,Composition
0,1B1,ChestWidth,,-1.00,1.00,Increm\nent,Deviation,Incremen\nt,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1X1,BottomHem Width,,-1.00,1.00,46.00,,50.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1F3,AcrossShoulder,,-1.00,1.00,36.00,,38.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1F6,ShoulderAngle,,-0.50,0.50,2.50,,2.50,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1G5,SleeveLength (Short),,-1.00,1.00,37.00,,39.00,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [25]:
# Save the data to an Excel file
output_file = "dynamic_output.xlsx"
df.to_excel(output_file, index=False)

print(f"Data has been successfully saved to {output_file}")


Data has been successfully saved to dynamic_output.xlsx


In [8]:
#dataCombining----------------------------------------------------------------------

In [ ]:
import pandas as pd

# Load the original Excel file containing all data
input_file = "dynamic_output.xlsx"  # Replace with the actual file path
data = pd.read_excel(input_file)

# Display the first few rows to understand the structure
data.head()


,Dim,Description,Comment,Tol (-),Tol (+),XS,S,M,L,XL
0,Dim,Description,Comment,Tol\n(-),Tol\n(+),XS,NaN,NaN,S,NaN
1,1B1,ChestWidth,NaN,-1.00,1.00,Increm\nent,Sam\nple,Devia\ntion,Incremen\nt,Sampl\ne
2,1X1,BottomHem Width,NaN,-1.00,1.00,46.00,NaN,NaN,50.00,NaN
3,1F3,AcrossShoulder,NaN,-1.00,1.00,36.00,NaN,NaN,38.00,NaN
4,1F6,ShoulderAngle,NaN,-0.50,0.50,2.50,NaN,NaN,2.50,NaN


In [10]:
# Identify rows for the first and second tables
table1_start = data.index[data['Dim'] == "Displaying 1-9 results"].tolist()
table2_start = data.index[data['Dim'] == "Displaying 10-15 results"].tolist()

# Extract the two tables based on the identified rows
if table1_start and table2_start:
    table1 = data.iloc[:table2_start[0]].reset_index(drop=True)
    table2 = data.iloc[table2_start[0]:].reset_index(drop=True)

# Drop rows that contain display headings
table1 = table1[table1['Dim'] != "Displaying 1-9 results"]
table2 = table2[table2['Dim'] != "Displaying 10-15 results"]


In [11]:
#----------------------------------------------------------------------------------

In [12]:
# Define paths
input_path = "dynamic_output.xlsx"  # Replace with actual file path
data = pd.read_excel(input_path)

# Process Table 1
table1 = data[data['Dim'] != "Displaying 1-9 results"]
table1 = table1.reset_index(drop=True)

# Process Table 2
table2 = data[data['Dim'] != "Displaying 10-15 results"]
table2 = table2.reset_index(drop=True)



In [20]:
print(table1)


                         Dim                    Description  \
0                        Dim                    Description   
1                        1B1                     ChestWidth   
2                        1X1                BottomHem Width   
3                        1F3                 AcrossShoulder   
4                        1F6                  ShoulderAngle   
5                        1G5           SleeveLength (Short)   
6                        1H2                   ArmholeDepth   
7                        2A1                 Upper ArmWidth   
8                        2K1                SleeveHem Width   
9                        3B1                  NecklineWidth   
10                         M                            NaN   
11               Incremen\nt                       Sampl\ne   
12                     54.00                            NaN   
13                     40.00                            NaN   
14                      2.50                           

In [21]:
print(table2)

                       Dim                    Description  \
0                      Dim                    Description   
1                      1B1                     ChestWidth   
2                      1X1                BottomHem Width   
3                      1F3                 AcrossShoulder   
4                      1F6                  ShoulderAngle   
5                      1G5           SleeveLength (Short)   
6                      1H2                   ArmholeDepth   
7                      2A1                 Upper ArmWidth   
8                      2K1                SleeveHem Width   
9                      3B1                  NecklineWidth   
10  Displaying 1-9 results                            NaN   
11                       M                            NaN   
12             Incremen\nt                       Sampl\ne   
13                   54.00                            NaN   
14                   40.00                            NaN   
15                    2.

In [ ]:
# Combine tables while ensuring column alignment
combined_table = pd.concat([table1, table2], ignore_index=True)

# Rearrange or rename columns if needed for consistency
combined_table.columns = [
    "Dim", "Description", "Tol (-)", "Tol (+)",
    "XS Increment", "XS Sample", "XS Deviation",
    "S Increment", "S Sample", "S Deviation",
    "M Increment", "M Sample", "M Deviation",
    "L Increment", "L Sample", "L Deviation",
    "XL Increment", "XL Sample", "XL Deviation"
]


ValueError: Length mismatch: Expected axis has 10 elements, new values have 19 elements

In [14]:
# Check the number of columns in table1 and table2
print("Table1 Columns:", table1.columns)
print("Table2 Columns:", table2.columns)

# Check the structure of combined_table
print("Combined Table Columns:", combined_table.columns)


Table1 Columns: Index(['Dim', 'Description', 'Comment', 'Tol (-)', 'Tol (+)', 'XS', 'S', 'M',
       'L', 'XL'],
      dtype='object')
Table2 Columns: Index(['Dim', 'Description', 'Comment', 'Tol (-)', 'Tol (+)', 'XS', 'S', 'M',
       'L', 'XL'],
      dtype='object')
Combined Table Columns: Index(['Dim', 'Description', 'Comment', 'Tol (-)', 'Tol (+)', 'XS', 'S', 'M',
       'L', 'XL'],
      dtype='object')


In [15]:
# Dynamically use the column names from the combined table
combined_table.columns = combined_table.columns  # This ensures no mismatch


In [16]:
# Check column counts
print("Table1 Column Count:", len(table1.columns))
print("Table2 Column Count:", len(table2.columns))
print("Combined Table Column Count:", len(combined_table.columns))


Table1 Column Count: 10
Table2 Column Count: 10
Combined Table Column Count: 10


In [17]:
# Ensure both tables have the same column names before concatenation
table1.columns = combined_table.columns
table2.columns = combined_table.columns

# Combine tables
combined_table = pd.concat([table1, table2], ignore_index=True)


In [ ]:
# Save the combined table to an Excel file
output_file = "final_combined_table_dynamic.xlsx"
combined_table.to_excel(output_file, index=False)

print(f"Combined table saved to {output_file}")


PermissionError: [Errno 13] Permission denied: 'final_combined_table_dynamic.xlsx'